In [11]:
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# col_names = ['temperature','occurrence_of_nausea','lumbar_pain','urine_pushing','micturition_pains','burning_of_urethra','decision_1','decision_2']
dataset_inflammation = pd.read_table('AcuteInflammations.data', sep=',')
print("\nDataset Acute Inflammations:")
dataset_inflammation


Dataset Acute Inflammations:


,temperature,occurrence_of_nausea,lumbar_pain,urine_pushing,micturition_pains,burning_of_urethra,decision_1,decision_2
0,35.5,0,1,0,0,0,0,0
1,35.9,0,0,1,1,1,1,0
2,35.9,0,1,0,0,0,0,0
3,36.0,0,0,1,1,1,1,0
4,36.0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
115,41.4,0,1,1,0,1,0,1
116,41.5,0,0,0,0,0,0,0
117,41.5,1,1,0,1,0,0,1
118,41.5,0,1,1,0,1,0,1


In [12]:
dataset_inflammation.columns

Index(['temperature', 'occurrence_of_nausea', 'lumbar_pain', 'urine_pushing',
       'micturition_pains', 'burning_of_urethra', 'decision_1', 'decision_2'],
      dtype='object')

In [13]:
inflammation_values = dataset_inflammation.iloc[:,0:8].values
print("\nAcute Inflammations features:\n")
print(inflammation_values)


Acute Inflammations features:

[[35.5  0.   1.   0.   0.   0.   0.   0. ]
 [35.9  0.   0.   1.   1.   1.   1.   0. ]
 [35.9  0.   1.   0.   0.   0.   0.   0. ]
 [36.   0.   0.   1.   1.   1.   1.   0. ]
 [36.   0.   1.   0.   0.   0.   0.   0. ]
 [36.   0.   1.   0.   0.   0.   0.   0. ]
 [36.2  0.   0.   1.   1.   1.   1.   0. ]
 [36.2  0.   1.   0.   0.   0.   0.   0. ]
 [36.3  0.   0.   1.   1.   1.   1.   0. ]
 [36.6  0.   0.   1.   1.   1.   1.   0. ]
 [36.6  0.   0.   1.   1.   1.   1.   0. ]
 [36.6  0.   1.   0.   0.   0.   0.   0. ]
 [36.6  0.   1.   0.   0.   0.   0.   0. ]
 [36.7  0.   0.   1.   1.   1.   1.   0. ]
 [36.7  0.   1.   0.   0.   0.   0.   0. ]
 [36.7  0.   1.   0.   0.   0.   0.   0. ]
 [36.8  0.   0.   1.   1.   1.   1.   0. ]
 [36.8  0.   0.   1.   1.   1.   1.   0. ]
 [36.9  0.   0.   1.   1.   1.   1.   0. ]
 [36.9  0.   1.   0.   0.   0.   0.   0. ]
 [37.   0.   0.   1.   1.   0.   1.   0. ]
 [37.   0.   0.   1.   1.   0.   1.   0. ]
 [37.   0.   1.   0.  

In [14]:
inflammation_normalizada = dataset_inflammation.values.copy()
normalizador =  StandardScaler()

inflammation_normalizada = normalizador.fit_transform(dataset_inflammation)
dataset_inflammation['temperature'] = inflammation_normalizada[:,0]

print("\nDataset Acute Inflammation normalizada:")
dataset_inflammation


Dataset Acute Inflammation normalizada:


,temperature,occurrence_of_nausea,lumbar_pain,urine_pushing,micturition_pains,burning_of_urethra,decision_1,decision_2
0,-1.779796,0,1,0,0,0,0,0
1,-1.558989,0,0,1,1,1,1,0
2,-1.558989,0,1,0,0,0,0,0
3,-1.503788,0,0,1,1,1,1,0
4,-1.503788,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
115,1.477107,0,1,1,0,1,0,1
116,1.532308,0,0,0,0,0,0,0
117,1.532308,1,1,0,1,0,0,1
118,1.532308,0,1,1,0,1,0,1


In [15]:
inflammation_classes = dataset_inflammation.iloc[:,7].values
print("\nAcute Inflammations classes:\n")
print(inflammation_classes)
print("\nAcute Inflammations classes shape:")
print(inflammation_classes.shape)


Acute Inflammations classes:

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 0 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 0 1 1 1]

Acute Inflammations classes shape:
(120,)


In [16]:
kf = model_selection.StratifiedKFold(n_splits = 2)
arvore_decisao = tree.DecisionTreeClassifier(criterion='entropy', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort='deprecated', ccp_alpha=0.0)
vizinhos_proximos = KNeighborsClassifier(n_neighbors=3, weights = 'distance')
naive_bayes_gaussian = GaussianNB()
regressao_logistica = LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='saga', max_iter=100, multi_class='auto', verbose=0, warm_start=False, n_jobs=None, l1_ratio=None)
rede_neural = MLPClassifier(hidden_layer_sizes=(5,), activation="logistic", max_iter= 300, alpha=0.001, solver="sgd", tol=1e-4, verbose=True, learning_rate_init=.01)

In [17]:
predicted_classes = dict()
predicted_classes['arvore_decisao'] = np.zeros(inflammation_classes.shape)
predicted_classes['vizinhos_proximos'] = np.zeros(inflammation_classes.shape)
predicted_classes['naive_bayes_gaussian'] = np.zeros(inflammation_classes.shape)
predicted_classes['regressao_logistica'] = np.zeros(inflammation_classes.shape)
predicted_classes['rede_neural'] = np.zeros(inflammation_classes.shape)

In [18]:

for train, test in kf.split(inflammation_values, inflammation_classes):
    data_train, target_train = inflammation_values[train], inflammation_classes[train]
    data_test, target_test = inflammation_values[test], inflammation_classes[test]

    arvore_decisao = arvore_decisao.fit(data_train, target_train)
    arvore_decisao_predicted = arvore_decisao.predict(data_test)
    predicted_classes['arvore_decisao'][test] = arvore_decisao_predicted

    vizinhos_proximos = vizinhos_proximos.fit(data_train, target_train)
    vizinhos_proximos_predicted = vizinhos_proximos.predict(data_test)
    predicted_classes['vizinhos_proximos'][test] = vizinhos_proximos_predicted

    naive_bayes_gaussian = naive_bayes_gaussian.fit(data_train, target_train)
    naive_bayes_gaussian_predicted = naive_bayes_gaussian.predict(data_test)
    predicted_classes['naive_bayes_gaussian'][test] = naive_bayes_gaussian_predicted

    regressao_logistica = regressao_logistica.fit(data_train, target_train)
    regressao_logistica_predicted = regressao_logistica.predict(data_test)
    predicted_classes['regressao_logistica'][test] = regressao_logistica_predicted

    rede_neural = rede_neural.fit(data_train, target_train)
    rede_neural_predicted = rede_neural.predict(data_test)
    predicted_classes['rede_neural'][test] = rede_neural_predicted

c:\users\david\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\david\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\users\david\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\linear_model\_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
c:\users\david\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't conver

Iteration 1, loss = 0.68974962
Iteration 2, loss = 0.68933206
Iteration 3, loss = 0.68872567
Iteration 4, loss = 0.68793792
Iteration 5, loss = 0.68697514
Iteration 6, loss = 0.68584921
Iteration 7, loss = 0.68459114
Iteration 8, loss = 0.68327225
Iteration 9, loss = 0.68202267
Iteration 10, loss = 0.68101530
Iteration 11, loss = 0.68038708
Iteration 12, loss = 0.68014649
Iteration 13, loss = 0.68017190
Iteration 14, loss = 0.68029988
Iteration 15, loss = 0.68040150
Iteration 16, loss = 0.68040262
Iteration 17, loss = 0.68027404
Iteration 18, loss = 0.68001866
Iteration 19, loss = 0.67966378
Iteration 20, loss = 0.67925493
Iteration 21, loss = 0.67884575
Iteration 22, loss = 0.67848284
Iteration 23, loss = 0.67819174
Iteration 24, loss = 0.67797224
Iteration 25, loss = 0.67780506
Iteration 26, loss = 0.67766368
Iteration 27, loss = 0.67752425
Iteration 28, loss = 0.67737076
Iteration 29, loss = 0.67719614
Iteration 30, loss = 0.67700098
Iteration 31, loss = 0.67679128
Iteration 32, los

In [19]:
for classificador in predicted_classes.keys():
    print("================================================================================================")
    print("Resultados do classificador %s\n%s\n"
          %(classificador, metrics.classification_report(inflammation_classes, predicted_classes[classificador])))
    print("Matriz de confusão: \n%s\n\n\n" % metrics.confusion_matrix(inflammation_classes, predicted_classes[classificador]))


Resultados do classificador arvore_decisao
              precision    recall  f1-score   support

           0       1.00      0.69      0.81        70
           1       0.69      1.00      0.82        50

    accuracy                           0.82       120
   macro avg       0.85      0.84      0.82       120
weighted avg       0.87      0.82      0.82       120


Matriz de confusão: 
[[48 22]
 [ 0 50]]



Resultados do classificador vizinhos_proximos
              precision    recall  f1-score   support

           0       0.91      0.86      0.88        70
           1       0.81      0.88      0.85        50

    accuracy                           0.87       120
   macro avg       0.86      0.87      0.86       120
weighted avg       0.87      0.87      0.87       120


Matriz de confusão: 
[[60 10]
 [ 6 44]]



Resultados do classificador naive_bayes_gaussian
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        70
           